##HW 01 - Line Search Optimization
***Tecnológico de Monterrey - MCCi - Evolutionary Computation Course***
*   Luis Felipe Brenes - A00819817
*   Diego Acosta Ugalde - A01367987

In this jupyter notebook we implement three diferent kinds of optimization methods that can be used to maximize or minimize mathematical function:


*   Line Search (using gradient descent with Wolfe's Conditions)
*   Line Search (using Newton's Method)
*   Hill Climber Optimization

Then these methods are compared taking into account the following measures:


*   **Correctness:** Does it find the global minimum? How big is the error between the returned and the expected output?
*   **Efficiency:** Does it find a good enough solution fast? How many iterations does it take for the algorithm to return an estimated minimum?



###**STEP 0: Importing needed modules**
*note: sympy is a library that enables python to compute math as symbolic expressions. You can find more about it here: https://www.sympy.org/en/index.html*

In [30]:
import time
start_time = time.time()

In [31]:
import sympy as sp
import numpy as np
from scipy.special import logsumexp
import math
import matplotlib.pyplot as plt
import random
import plotly.graph_objects as go

x, y = sp.symbols('x y') #defines x y as the symbols to be used in sympy

In [32]:
c_nabla = 0 #Iteration counter. Counts how many times the gradient is calculated.

###**STEP 1: Creating needed functions**







####Group 1: Functions for mathematical and vector operations

*   **nabla**(function): returns first derivative
*   **unit_vect**(vector): returns unit vector
*   **prodScalarVect**(scalar,vector): returns the product between a scalar and a vector.
*   **sum2vect**(vector1,vector2): returns the sum of vector1 and vector2.
*   **amplitud_vect**(vector): returns the magnitude of a vector.
*   **evaluate_f**(function, X): returns function evaluated in point X

In [33]:
def nabla(function: sp.core.add.Add) -> list:
    decision_variables = [x,y]

    global c_nabla
    c_nabla+=1

    # diff calculates the derivative.
    return [sp.diff(function, decision_variable) for decision_variable in decision_variables]

In [34]:
def unitVect(v):
  return v / np.linalg.norm(v)

In [35]:
#Vector operations

from operator import add
def prodScalarVect(S, V):
  return [comp * S for comp in V]

def sum2vect(V1,V2):
  return list(map(add, V1, V2))

def amplitud_vect(vect: list, eval_point:list) -> float:
  eval_vect = map(lambda x: x.evalf(subs={'x':eval_point[0], 'y':eval_point[1]}),vect)
  pot_eval_vect = map(lambda x: x**2, eval_vect)
  amp_vect = math.sqrt(sum(pot_eval_vect))
  return amp_vect

In [36]:
def extract_symbols_from_function(f: sp.core.add.Add):
    """
        Returns a vector with the symbols of a given function sorted by their name
        @param f : function whose symbols will be extracted
        @returns a list with the symbols of f sorted by their name
    """
    return sorted(f.free_symbols, key=lambda s: s.name)


def evaluate_f(f: sp.core.add.Add, x: list) -> float:
    """
        Evaluate a given function in a given coordinates
        @param f : function to be evaluated
        @param x : a list to evaluate f in
        @return f(x) which is a scalar resulting of evaluating f in x
    """
    # First exctract symbols of the function and sort them by name
    decision_variables = extract_symbols_from_function(f)
    # Then calculate the function evaluated in requested values
    zipped_variables_and_their_values = dict(zip(decision_variables, x))
    evaluation = f.evalf(subs=zipped_variables_and_their_values, chop=True)
    # Convert the result to float and round it to two decimals
    #return round(float(evaluation), 2)
    return(float(evaluation))

####Group 2: Functions to calculate search direction p

*   **gradient_descent**(function): returns search direction p using gradient descent.
*   **newton_method**(function): returns search direction p using gradient descent.

In [37]:
def gradient_descent(objF, eval_p, minmax=[-1,1]):
  return unitVect(list(map(lambda f: minmax * evaluate_f(f,eval_p), nabla(objF))))
  #return list(map(lambda f: minmax * evaluate_f(f,eval_p), nabla(objF)))

In [38]:
def newton_method(objF, eval_p, minmax=[-1,1]):
    hess = [nabla(comp) for comp in nabla(objF)]

    # Evaluar el hessiano
    ev_hess = []
    for comp in hess:
      hess_evaluated = [evaluate_f(e, eval_p) for e in comp]
      ev_hess.append(hess_evaluated)

    #invertir hessiano
    inverseHess = np.linalg.inv(ev_hess)

    #evaluar nabla
    ev_nabla = [evaluate_f(comp, eval_p) for comp in nabla(objF)]

    return(minmax*np.matmul(inverseHess, ev_nabla))

####Group 3: Functions to calculate step size t.

*   **wolfeCondition1**(function, X, p, t, c1): returns bool (True/False) if Wolfe Condition 1 is met.

*   **wolfeCondition2**(function, X, p, t, c2): returns bool (True/False) if Wolfe Condition 2 is met.

* **stepSize**(function, X, p, t): returns step size t, using Wolfe's Conditions.

*note: some literature refers to step size as alpha.*

In [39]:
def wolfeCondition1(f, X, p, t, c1):

  #ineqL = evaluate_f( f, sum2vect(X,prodScalarVect(t,p)))
  ineqL  = f.evalf(subs={'x': sum2vect(X,prodScalarVect(t,p))[0], 'y':sum2vect(X,prodScalarVect(t,p))[1]},chop=True)

  #ineqR = evaluate_f(f,X) + c1 * t *  np.matmul([evaluate_f(comp, X) for comp in nabla(f)] ,np.transpose(p))
  ineqR = f.evalf(subs={'x': X[0], 'y':X[1]},chop=True)+ + c1 * t *  np.matmul([comp.evalf(subs={'x': X[0], 'y':X[1]}) for comp in nabla(f)] ,np.transpose(p))

  return  ineqL > ineqR

In [40]:
def wolfeCondition2(f, X, p, t, c2):
  #ineqL = np.matmul ([evaluate_f(comp, sum2vect(X, prodScalarVect(t,p))) for comp in nabla(f)], np.transpose(p))
  ineqL = np.matmul ([comp.evalf(subs={'x': sum2vect(X, prodScalarVect(t,p))[0], 'y':sum2vect(X, prodScalarVect(t,p))[1]},chop=True) for comp in nabla(f)], np.transpose(p))

  #ineqR = c2 * np.matmul([evaluate_f(comp, X) for comp in nabla(f)] ,np.transpose(p))
  ineqR = c2 * np.matmul([comp.evalf(subs={'x':X[0], 'y': X[1]},chop=True)for comp in nabla(f)] ,np.transpose(p))
  return  ineqL < ineqR

In [41]:
def stepSize(f, X, p, t):
  c1 = 0.0001
  c2 = 0.9
  flag = True
  while flag:
    if wolfeCondition1(f, X, p, t, c1):
      t/=2
      continue
    elif wolfeCondition2(f, X, p, t, c2):
      t*=2
    else:
      flag = False
  return t

###**STEP 2.1: Hill Climber Method**
This functions is the implementation for Hill Climber Method.

*   **ensure_xbest**(x_best, f_obj): returns the best point that the hill climber method could find in the number of iterations given (iter).



In [42]:
p = 5 # Open ball radius

In [43]:
iter = 2000 # Max number of iterations

In [44]:
def ensure_xbest(x_best, f_ob):
  print('Start point:', x_best)
  i = 0
  i_best = 0
  tolerance = .001
  while i < iter:
    # Generate new x in circle
    angle = random.uniform(0, 2*math.pi)
    distance = random.uniform(0, p)
    x = x_best[0] + distance * math.cos(angle)
    y = x_best[1] + distance * math.sin(angle)
    x_new = [x,y]
    # print('X new: ', x_new)
    x_eval = f_ob.evalf(subs={'x': x_new[0], 'y': x_new[1]})
    # print('X new:', round(float(x_eval), 2))
    x_best_eval = f_ob.evalf(subs={'x': x_best[0], 'y': x_best[1]})
    # print('X best:', round(float(x_best_eval)))
    if (x_eval < x_best_eval):
      # print('X new evaluation:', round(float(x_eval), 2))
      if abs(x_best_eval-x_eval) <= tolerance:
        # No important improvement
        break
      x_best = x_new
      # print('X new best:', x_best)
      i_best += 1
    i += 1
  # print('--------------------------------------')
  # print('\n\nNumber of iterations: ', i)
  # print('Number of best X found: ', i_best)

  return x_best, i

###**STEP 2.2: Line Search Methods**
These two functions are the implementation of Line Search Method.

One uses gradient descent and Wolfe's Conditions

####**line_search_grad**(objF, tol, startX, minmax=[-1,1])

* Performs line search on an objective function using gradient descent and Wolfe's Conditions.
* Takes as arguments *objF (objective function)*, *tol (tolerance)*, *startX (starting point)*, *minmax (1:maximixes, -1:minimizes)*
* Returns estimated minimum or maximum of the objectife function.

####**line_search_newt**(objF, tol, startX, minmax=[-1,1])

* Performs line search on an objective function using gradient descent and Wolfe's Conditions.
* Takes as arguments *objF (objective function)*, *tol (tolerance)*, *startX (starting point)*, *minmax (1:maximixes, -1:minimizes)*
* Wolfe's Conditions are not used. **p** is used to calculate the Line Search on it's own.
* Returns estimated minimum or maximum of the objectife function.



In [45]:
def line_search_grad (objF, tol, startX, minmax=[-1,1]):
  X = startX
  c = 0 #iteration counter
  t = 5 #initial step size

  while amplitud_vect(nabla(objF), X) >= tol:
    #print("X =", X, "t= ",t)
    p_gd = gradient_descent(objF,X,minmax) #p using gradient descent

    t = stepSize(objF, X, p_gd, t) #VAMOS EN ESTE PASO
    Xnew = sum2vect(X, prodScalarVect(t,p_gd))
    if X == Xnew:
      break
    X = Xnew
    c+=1
    if c+c_nabla>= 1000: #HERE YOU CAN CHANGE ITERATION LIMIT
      print("\n\nITERATION LIMIT REACHED")
      return X,c
  return X, c

In [46]:
def line_search_newt (objF, tol, startX, minmax=[-1,1]):
  X = startX
  c = 0 #iteration counter
  t = 1 #initial step size

  while amplitud_vect(nabla(objF), X) >= tol:
    #print("X =", X, "t= ",t)
    p = newton_method(objF,X,minmax) #p using newton method

    #t = stepSize(objF, X, p, t, tol)
    Xnew = sum2vect(X, prodScalarVect(t,p))
    if X == Xnew:
      break
    X = Xnew
    c+=1
    if c+c_nabla>= 1000: #HERE YOU CAN CHANGE ITERATION LIMIT
      print("\n\nITERATION LIMIT REACHED")
      return X,c
  return X, c

###**STEP 4: Performance Evaluation**
Evaluates performance of different methods printing: **returned optimal**, **real optimal**, **error** calculated using the 2-norm, and **number of iterations** each method needed to calculate the optimal. The number of iterations takes into account each main function iteration and each gradient and hessian calculated.




####**Hill Climber evaluation**

In [47]:
x, y = sp.symbols('x y')

optimal = [2,1]
start_point = [-2,2]
F1 = -1*(2*x*y + 2*x - x**2 - 2*y**2)
x_best1, c = ensure_xbest(start_point, F1)
print('FUNCTION #1')
print('------------------------------------------------------------')
print('Returned Optimal: ', x_best1)
print('    Real Optimal: ', optimal)
print('    2-Norm Error: ', round(np.linalg.norm(np.array(x_best1)-np.array(optimal)),4))
print('      Iterations: ', c,'\n\n')

optimal = [1,1]
start_point = [-1.5,1.5]
F2 = 100*(x**2 - y)**2 + (1 - x)**2
x_best2, c = ensure_xbest(start_point, F2)
print('FUNCTION #2')
print('------------------------------------------------------------')
print('Returned Optimal: ', x_best2)
print('    Real Optimal: ', optimal)
print('    2-Norm Error: ', round(np.linalg.norm(np.array(x_best2)-np.array(optimal)),4))
print('      Iterations: ', c,'\n\n')

optimal = [0,0]
start_point = [-2,2]
F3 = 20 + (x**2 - 10 * sp.cos(2 * sp.pi * x)) + (y**2 - 10 * sp.cos(2 * sp.pi * y))
x_best3, c = ensure_xbest(start_point, F3)
print('FUNCTION #3')
print('------------------------------------------------------------')
print('Returned Optimal: ', x_best3)
print('    Real Optimal: ', optimal)
print('    2-Norm Error: ', round(np.linalg.norm(np.array(x_best3)-np.array(optimal)),4))
print('      Iterations: ', c,'\n\n')

Start point: [-2, 2]
FUNCTION #1
------------------------------------------------------------
Returned Optimal:  [1.8736939491821842, 0.9164589136067106]
    Real Optimal:  [2, 1]
    2-Norm Error:  0.1514
      Iterations:  508 


Start point: [-1.5, 1.5]
FUNCTION #2
------------------------------------------------------------
Returned Optimal:  [1.0325208400600356, 1.0518012097895877]
    Real Optimal:  [1, 1]
    2-Norm Error:  0.0612
      Iterations:  2000 


Start point: [-2, 2]
FUNCTION #3
------------------------------------------------------------
Returned Optimal:  [-0.00574771897050937, -0.9975452531588039]
    Real Optimal:  [0, 0]
    2-Norm Error:  0.9976
      Iterations:  2000 




####**Line Search using Newton's Method and a fixed t value on 1**

In [48]:
c_nabla = 0
F1 = -2*x*y - 2*x + x**2 + 2*y**2 # minimo debería ser [2,1]
tol1 = 0.1
X1 = [-2,2]
optimal = [2,1]
ls_newton1, c = line_search_newt(F1, tol1, X1, -1)

print('FUNCTION #1')
print('------------------------------------------------------------')
print('Returned Optimal: ', ls_newton1)
print('    Real Optimal: ', optimal)
print('    2-Norm Error: ', round(np.linalg.norm(np.array(ls_newton1)-np.array(optimal)),4))
print('      Iterations: ', c+c_nabla,'\n\n')


c_nabla = 0
F2 = 100 * (x**2-y)**2 + (1-x)**2 # minimo debería ser [1,1]
tol2 = 0.1
X2 = [-1.5,1.5]
optimal = [1,1]
ls_newton2, c = line_search_newt(F2, tol2, X2, -1)

print('FUNCTION #2')
print('------------------------------------------------------------')
print('Returned Optimal: ', ls_newton2)
print('    Real Optimal: ', optimal)
print('    2-Norm Error: ', round(np.linalg.norm(np.array(ls_newton2)-np.array(optimal)),4))
print('      Iterations: ', c+c_nabla,'\n\n')

c_nabla = 0
F3 = 20 + (x**2 - 10 * sp.cos(2 * sp.pi * x)) + (y**2 - 10 * sp.cos(2 * sp.pi * y)) # mínimo debería ser [0,0]
tol3 = 0.1
X3 = [-2,2]
optimal = [0,0]
ls_newton3, c = line_search_newt(F3, tol3, X3, -1)


print('FUNCTION #3')
print('------------------------------------------------------------')
print('Returned Optimal: ', ls_newton3)
print('    Real Optimal: ', optimal)
print('    2-Norm Error: ', round(np.linalg.norm(np.array(ls_newton3)-np.array(optimal)),4))
print('      Iterations: ', c+c_nabla,'\n\n')


FUNCTION #1
------------------------------------------------------------
Returned Optimal:  [2.0, 1.0]
    Real Optimal:  [2, 1]
    2-Norm Error:  0.0
      Iterations:  7 


FUNCTION #2
------------------------------------------------------------
Returned Optimal:  [0.9999997317006529, 0.9999994634013711]
    Real Optimal:  [1, 1]
    2-Norm Error:  0.0
      Iterations:  31 


FUNCTION #3
------------------------------------------------------------
Returned Optimal:  [-1.9899122337085495, 2.0100877662914503]
    Real Optimal:  [0, 0]
    2-Norm Error:  2.8285
      Iterations:  23 




####**Line Search using Gradient Descent and Wolfe's Conditions**
Iteration Limit is set to 30000 on reported results. It takes around 5-10min to run this cell. You can reduce iteration limit altering that line of code **line_search_grad**() and **line_search_newt**().

Iteration limit is currently set to **1000** in order to make it return a result faster.

In [49]:
c_nabla = 0
F1 = -2*x*y - 2*x + x**2 + 2*y**2 # minimo debería ser [2,1]
tol1 = 0.1
X1 = [-2,2]
optimal = [2,1]
ls_grad1, c = line_search_grad(F1, tol1, X1, -1)

print('FUNCTION #1')
print('------------------------------------------------------------')
print('Returned Optimal: ', ls_grad1)
print('    Real Optimal: ', optimal)
print('    2-Norm Error: ', round(np.linalg.norm(np.array(ls_grad1)-np.array(optimal)),4))
print('      Iterations: ', c+c_nabla,'\n\n')


c_nabla = 0
F2 = 100 * (x**2-y)**2 + (1-x)**2 # minimo debería ser [1,1]
tol2 = 0.1
X2 = [-1.5,1.5]
optimal = [1,1]
ls_grad2, c = line_search_grad(F2, tol2, X2, -1)

print('FUNCTION #2')
print('------------------------------------------------------------')
print('Returned Optimal: ', ls_grad2)
print('    Real Optimal: ', optimal)
print('    2-Norm Error: ', round(np.linalg.norm(np.array(ls_grad2)-np.array(optimal)),4))
print('      Iterations: ', c+c_nabla,'\n\n')

c_nabla = 0
#F3 = 20 + (x*2 - 10 * sp.cos(2 * sp.pi * x)) + (y*2 - 10 * sp.cos(2 * sp.pi * y)) # mínimo debería ser [0,0]
F3 = 20 + (x**2 - 10 * sp.cos(2 * sp.pi * x)) + (y**2 - 10 * sp.cos(2 * sp.pi * y))
tol3 = 0.1
X3 = [-2,2]
optimal = [0,0]
ls_grad3, c = line_search_grad(F3, tol3, X3, -1)


print('FUNCTION #3')
print('------------------------------------------------------------')
print('Returned Optimal: ', ls_grad3)
print('    Real Optimal: ', optimal)
print('    2-Norm Error: ', round(np.linalg.norm(np.array(ls_grad3)-np.array(optimal)),4))
print('      Iterations: ', c+c_nabla,'\n\n')

FUNCTION #1
------------------------------------------------------------
Returned Optimal:  [1.9334044105373784, 0.97465703599302]
    Real Optimal:  [2, 1]
    2-Norm Error:  0.0713
      Iterations:  56 




ITERATION LIMIT REACHED
FUNCTION #2
------------------------------------------------------------
Returned Optimal:  [-1.0856085567251808, 1.1754716212949166]
    Real Optimal:  [1, 1]
    2-Norm Error:  2.093
      Iterations:  1005 




ITERATION LIMIT REACHED
FUNCTION #3
------------------------------------------------------------
Returned Optimal:  [-1.999999990481965, 2.000000009518]
    Real Optimal:  [0, 0]
    2-Norm Error:  2.8284
      Iterations:  1002 




In [50]:
print('Time: %s seconds' %(time.time() - start_time))

Time: 15.892969131469727 seconds
